In [1]:
import numpy as np

In [8]:
np.linspace(80, 120, 8 + 1)

array([ 80.,  85.,  90.,  95., 100., 105., 110., 115., 120.])

In [15]:
np.linspace(50, 150, 50+1)

array([ 50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,  66.,  68.,  70.,
        72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,  88.,  90.,  92.,
        94.,  96.,  98., 100., 102., 104., 106., 108., 110., 112., 114.,
       116., 118., 120., 122., 124., 126., 128., 130., 132., 134., 136.,
       138., 140., 142., 144., 146., 148., 150.])

In [16]:
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed

In [17]:
n_cpu_cores = multiprocessing.cpu_count()  # get the number of CPU cores
max_threads_cpu_task = n_cpu_cores - 2  # leave 2 cores free for other tasks

In [18]:
max_threads_cpu_task

18

In [12]:
from itertools import product

In [21]:
Option_types=["call", "put"]  # call, put
S_values=list(np.linspace(50, 150, 50 + 1))  # 50, 52, 54, ..., 150
K_values=[100]  # 100
T_values=[
    0.08,
    0.25,
    0.5,
    1.0,
    1.5,
    2.0,
    2.5,
    3.0,
    4.0,
    5.0,
]  # 1 month, 3 months, 6 months, 1 year, 1.5 years, 2 years, 2.5 years, 3 years, 4 years, 5 years
r_values=[0.01, 0.025, 0.05, 0.075, 0.1]  # 1%, 2.5%, 5%, 7.5%, 10%
sigma_values=[0.1, 0.2, 0.4, 0.6, 0.8]
q_values=[0.0, 0.01, 0.02, 0.03, 0.04, 0.05]

In [10]:
Option_types=["call", "put"]
S1_values=list(np.linspace(80, 120, 8 + 1))
S2_values=list(np.linspace(80, 120, 8 + 1))
K1_values=[100]
K2_values=[100]
sigma1_values=[0.1, 0.2, 0.4, 0.6, 0.8],
sigma2_values=[0.1, 0.2, 0.4, 0.6, 0.8],
q1_values=[0.01, 0.02, 0.05]
q2_values=[0.01, 0.02, 0.05]
corr_values=[0.01, 0.2, 0.5, 0.8]
T_values=[0.08, 0.25, 0.5, 1.0, 1.5, 2.0, 5.0]
r_values=[0.01, 0.025, 0.05, 0.075, 0.1]

In [13]:
n = sum(
    1
    for _ in product(
        Option_types,
        S1_values,
        S2_values,
        K1_values,
        K2_values,
        sigma1_values,
        sigma2_values,
        q1_values,
        q2_values,
        corr_values,
        T_values,
        r_values,
    )
)
print("Number of combinations:", n)

Number of combinations: 204120


In [23]:
import math

lengths = [
    len(Option_types),
    len(S_values),
    len(K_values),
    len(T_values),
    len(r_values),
    len(sigma_values),
    len(q_values),
]

n_combinations = math.prod(lengths)
print("Number of combinations:", n_combinations)

Number of combinations: 153000


In [24]:
lengths

[2, 51, 1, 10, 5, 5, 6]

In [14]:
import pandas as pd

In [31]:
df = pd.read_csv("../data/options_dataset.csv")

In [34]:
df

,Option_type,S,K,T,r,sigma,q,bs_price,mc_price
0,call,50.0,100,0.08,0.01,0.1,0.00,1.026955e-133,0.000000
1,call,50.0,100,0.08,0.01,0.1,0.01,5.122990e-134,0.000000
2,call,50.0,100,0.08,0.01,0.1,0.02,2.553578e-134,0.000000
3,call,50.0,100,0.08,0.01,0.1,0.03,1.271828e-134,0.000000
4,call,50.0,100,0.08,0.01,0.1,0.04,6.329384e-135,0.000000
...,...,...,...,...,...,...,...,...,...
152995,put,150.0,100,5.00,0.10,0.4,0.05,9.607684e+00,9.582241
152996,put,150.0,100,5.00,0.10,0.8,0.03,2.920857e+01,29.189305
152997,put,150.0,100,5.00,0.10,0.8,0.02,2.860204e+01,28.554906
152998,put,150.0,100,5.00,0.10,0.8,0.05,3.042100e+01,30.509238


In [35]:
df.round(4)

,Option_type,S,K,T,r,sigma,q,bs_price,mc_price
0,call,50.0,100,0.08,0.01,0.1,0.00,0.0000,0.0000
1,call,50.0,100,0.08,0.01,0.1,0.01,0.0000,0.0000
2,call,50.0,100,0.08,0.01,0.1,0.02,0.0000,0.0000
3,call,50.0,100,0.08,0.01,0.1,0.03,0.0000,0.0000
4,call,50.0,100,0.08,0.01,0.1,0.04,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...
152995,put,150.0,100,5.00,0.10,0.4,0.05,9.6077,9.5822
152996,put,150.0,100,5.00,0.10,0.8,0.03,29.2086,29.1893
152997,put,150.0,100,5.00,0.10,0.8,0.02,28.6020,28.5549
152998,put,150.0,100,5.00,0.10,0.8,0.05,30.4210,30.5092


# EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("../data/options_dataset.csv")

In [3]:
df.describe()

,S,K,T,r,sigma,q,bs_price,mc_price
count,153000.000000,153000.0,153000.000000,153000.00000,153000.000000,153000.000000,1.530000e+05,153000.000000
mean,100.000000,100.0,1.983000,0.05200,0.420000,0.025000,2.340649e+01,23.406672
std,29.439299,0.0,1.565125,0.03265,0.256126,0.017078,1.971527e+01,19.716090
min,50.000000,100.0,0.080000,0.01000,0.100000,0.000000,3.147372e-135,0.000000
25%,74.000000,100.0,0.500000,0.02500,0.200000,0.010000,5.232921e+00,5.232654
50%,100.000000,100.0,1.750000,0.05000,0.400000,0.025000,2.066067e+01,20.664333
75%,126.000000,100.0,3.000000,0.07500,0.600000,0.040000,3.746151e+01,37.460397
max,150.000000,100.0,5.000000,0.10000,0.800000,0.050000,1.167373e+02,114.659695


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153000 entries, 0 to 152999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Option_type  153000 non-null  object 
 1   S            153000 non-null  float64
 2   K            153000 non-null  int64  
 3   T            153000 non-null  float64
 4   r            153000 non-null  float64
 5   sigma        153000 non-null  float64
 6   q            153000 non-null  float64
 7   bs_price     153000 non-null  float64
 8   mc_price     153000 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 10.5+ MB


In [ ]:
fig = plt.figure(figsize=(40, 40))

In [38]:
sns.pairplot(df, hue="Option_type", diag_kind="kde", height=2.5)

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x0000019B64446290> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt

